## Install and load modules

Zipline only works with an older version of pandas. It is necessary to downgrade to the version of pandas that is 0.22.0. We also need to download zipline.

In [5]:
!pip install quantopian

  ERROR: Could not find a version that satisfies the requirement quantopian (from versions: none)
ERROR: No matching distribution found for quantopian


In [ ]:
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import zipline
import numpy as np
from zipline.api import order, record, symbol
from zipline.assets import Equity, Future
from zipline import run_algorithm
from collections import OrderedDict
from datetime import datetime


## Load and Prelimenary Analysis of Data

In [ ]:
# Stock in our portfolio
stocks  = ["AAPL", "AMZN", "BA", "FB", "GOOG", "MA", "MSFT", "NVDA", "UNH", "V"]

# Calculate the weekly and yearly returns for stocks
stockList = []
# Creates a dataframe from CSV and saves to a list
for stock in stocks:
    df = pd.read_csv("{}.csv".format(stock), index_col =0, parse_dates=['Date'])
    stockList.append(df)   
# Creates a dataframe from the list of dataframes
stockdf = pd.concat(stockList, keys=stocks)
stockdf = stockdf.reset_index()
stockdf.rename(index=str, columns={"level_0":"ticker"}, inplace=True)
# Impute missing values
stockdf.head(5)

In [ ]:
# To computer the covariance matrix , turn data into long format with pivot table
datawide = pd.pivot_table(stockdf, values='adj close', index=['Date'], columns='ticker')
returns = np.log(datawide).diff()
returns.head(3)

## Make data compatible with Zipline

In [ ]:
# Store the different datasets
data = {}

# Iterate to load stock data from csv
for stock in stocks:
    data[stock] = pd.read_csv("{}.csv".format(stock), index_col =0, parse_dates=['Date'])
    data[stock] = data[stock][["open","high","low","close","adj close","volume"]]
    data[stock] = data[stock].resample("1d").mean()
    data[stock].fillna(method='ffill', inplace = True)

In [ ]:
# Convert data to panel
StockPrice = pd.Panel(data)
StockPrice.minor_axis = ["open","high","low","close","adj close","volume"]
StockPrice.major_axis = StockPrice.major_axis.tz_localize(pytz.utc)
print(StockPrice)

In [ ]:
plt.figure(figsize=(15,10))
for stock in stocks:
    data[stock]['adj close'].plot()
plt.legend(stocks)
plt.ylabel("Price in $")
data['GOOG']['close']

In [ ]:
# I dont think this does anything
%load_ext zipline

## Set constraints 

In [ ]:
# Constraints for our algorithm
initial_capital = 10000
# Time frame for our algorithm
startDate = datetime(2018,1,1,0,0,0,0,pytz.utc)
endDate = datetime(2018,12,1,0,0,0,0,pytz.utc)
# Trading calendar used for algorithm
tradingCalendar = "NYSE"


## Run Algorithm

In [ ]:
# Define necessary two functions to run algorithm
def initialize(context):
    context.assest = symbol('GOOG')
    
def handle_data(context, data):
    order(symbol('GOOG'),10)
    record(GOOG = data.current(symbol('GOOG'),'price'))



In [ ]:
perf = zipline.run_algorithm(start= startDate,
                            end= endDate,
                            initialize=initialize,
                            data_frequency='daily',
                            capital_base= initial_capital,
                            handle_data=handle_data,
                            data=StockPrice)
perf.head(5)
perf['benchmark_period_return'].agg(sum)